In [1]:

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from sklearn.metrics import f1_score, confusion_matrix
from tqdm import tqdm
import os


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [19]:
dataset_path = "C:/Users/vivek/Desktop/training/dataset" 

train_dataset = ImageFolder(f"{dataset_path}/train", transform=transform)
val_dataset   = ImageFolder(f"{dataset_path}/validate", transform=transform)
test_dataset = ImageFolder(f"{dataset_path}/test",transform=transform)

BATCH_SIZE = 32

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

test_loader=DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0
)
print("Train samples:", len(train_dataset))
print("Val samples:", len(val_dataset))
print("Test samples:",len(test_dataset))


Train samples: 17122
Val samples: 16968
Test samples: 16978


In [6]:
model = models.efficientnet_b0(pretrained=True)

in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, 2)

model = model.to(device)


In [7]:
for param in model.features.parameters():
    param.requires_grad = False


In [8]:
# Unfreeze last 4 EfficientNet blocks
for block in model.features[-4:]:
    for param in block.parameters():
        param.requires_grad = True


In [9]:
trainable = sum(p.requires_grad for p in model.parameters())
total = sum(1 for _ in model.parameters())

print(f"Trainable tensors: {trainable}")
print(f"Total tensors: {total}")


Trainable tensors: 109
Total tensors: 213


In [10]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-5
)


In [11]:
EPOCHS = 5

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader, desc=f"Phase 2 Epoch {epoch+1}/{EPOCHS}"):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"[Phase 2] Epoch {epoch+1} | Train Loss: {running_loss / len(train_loader):.4f}")


Phase 2 Epoch 1/5: 100%|██████████| 536/536 [07:12<00:00,  1.24it/s]


[Phase 2] Epoch 1 | Train Loss: 0.4983


Phase 2 Epoch 2/5: 100%|██████████| 536/536 [08:49<00:00,  1.01it/s]


[Phase 2] Epoch 2 | Train Loss: 0.2604


Phase 2 Epoch 3/5: 100%|██████████| 536/536 [09:16<00:00,  1.04s/it]


[Phase 2] Epoch 3 | Train Loss: 0.1850


Phase 2 Epoch 4/5: 100%|██████████| 536/536 [03:52<00:00,  2.31it/s]


[Phase 2] Epoch 4 | Train Loss: 0.1529


Phase 2 Epoch 5/5: 100%|██████████| 536/536 [00:54<00:00,  9.88it/s]

[Phase 2] Epoch 5 | Train Loss: 0.1338


In [13]:
model.eval()

all_preds = []
all_labels = []
val_loss = 0.0

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        val_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("Validation Loss:", val_loss / len(val_loader))


Validation Loss: 0.11778445333231893


In [14]:
f1 = f1_score(all_labels, all_preds)
cm = confusion_matrix(all_labels, all_preds)

print("Phase 2 F1-score:", f1)
print("Confusion Matrix:\n", cm)


Phase 2 F1-score: 0.9600188423717836
Confusion Matrix:
 [[8137  447]
 [ 232 8152]]


In [20]:
model.eval()

all_preds_test = []
all_labels_test = []
test_loss_test = 0.0

with torch.no_grad():
    for images, labels in test_loader:
        images_1 = images.to(device)
        labels_1 = labels.to(device)

        outputs_1 = model(images_1)
        loss_1 = criterion(outputs_1, labels_1)

        test_loss_test += loss_1.item()
        preds_1 = torch.argmax(outputs_1, dim=1)

        all_preds_test.extend(preds_1.cpu().numpy())
        all_labels_test.extend(labels_1.cpu().numpy())

print("test Loss:", test_loss_test / len(test_loader))


test Loss: 0.11863945291347788


In [21]:
f1_test = f1_score(all_labels_test, all_preds_test)
cm_test = confusion_matrix(all_labels_test, all_preds_test)

print("Phase 2 F1-score test:", f1_test)
print("Confusion Matrix test:\n", cm_test)


Phase 2 F1-score test: 0.9599481682176935
Confusion Matrix test:
 [[8149  440]
 [ 240 8149]]


In [22]:
os.makedirs("checkpoints", exist_ok=True)

torch.save(
    {
        "model_state_dict": model.state_dict(),
        "f1_score": f1
    },
    "checkpoints/best one .pth"
)

print("best one  checkpoint saved")

best one  checkpoint saved
